# Normal Day Analysis

In [1]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

In [2]:
datafile='data7.parquet'

In [3]:
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.


In [4]:
data=pd.read_parquet(datafile)

In [7]:
data.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709948,1232265,192.168.107.196,157.240.212.35,tcp,443,13896,113280
709949,1232398,192.168.107.196,157.240.212.35,tcp,443,45276,419472


In [8]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET,axis=1)]

private_dst_normal.head()


,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [9]:
grp_private_normal = private_dst_normal.groupby(["proto","port"])['dst_ip'].apply(list)

In [56]:

https_servers = set(grp_private_normal[('tcp', 443)])
print()

dns_servers =  set(grp_private_normal[('udp', 53)])
print()

## Servers 
tcp 443 {'192.168.107.223', '192.168.107.238', '192.168.107.222', '192.168.107.226'} https

udp 53 {'192.168.107.227', '192.168.107.234'} dns


In [57]:
grouped = data.groupby(['src_ip','dst_ip']).sum()

# Calculate the ratio
grouped['ratio'] = grouped['up_bytes'] / grouped['down_bytes']
grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()

timestamp  \
src_ip          dst_ip                      
192.168.107.100 104.16.249.130   94725010   
                104.22.48.100    36323031   
                104.244.42.129  246717524   
                104.26.13.207   100077577   
                104.26.14.51     79996025   
...                                   ...   
192.168.107.99  66.22.229.4      11237553   
                66.254.114.41    10868618   
                88.157.217.145   66107954   
                88.157.217.146  120743878   
                97.107.142.234   30141613   

                                                                            proto  \
src_ip          dst_ip                                                              
192.168.107.100 104.16.249.130  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.22.48.100                         tcptcptcptcptcptcptcptcptcp   
                104.244.42.129  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.26.13.207   tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.26.14.51     tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcptcp   
...                                                                           ...   
192.168.107.99  66.22.229.4                                             tcptcptcp   
                66.254.114.41                                           tcptcptcp   
                88.157.217.145  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                88.157.217.146  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                97.107.142.234                           tcptcptcptcptcptcptcptcp   

                                 port  up_bytes  down_bytes     ratio  
src_ip          dst_ip                                                 
192.168.107.100 104.16.249.130   7974    209457     1727986  0.121215  
                104.22.48.100    3987     93381      814578  0.114637  
                104.244.42.129  21707    512007     4246664  0.120567  
                104.26.13.207    8860    270821     6675043  0.040572  
                104.26.14.51     7088    205239     1890983  0.108536  
...                               ...       ...         ...       ...  
192.168.107.99  66.22.229.4      1329     23809      267000  0.089172  
                66.254.114.41    1329     37602      337169  0.111523  
                88.157.217.145   7974    159654     1506733  0.105960  
                88.157.217.146  14619    379690     4234007  0.089676  
                97.107.142.234   3544     71568      677703  0.105604  

[27120 rows x 6 columns]

In [58]:
mean_ratios = grouped.groupby('src_ip')['ratio'].mean().reset_index()

In [62]:
server_traffic = data[data.apply(lambda x: x['dst_ip'] in dns_servers or x['dst_ip'] in https_servers,axis=1)]


In [63]:
server_traffic.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [66]:
servers_grouped = server_traffic.groupby(['dst_ip','src_ip']).sum()

# Calculate the ratio
servers_grouped['ratio'] = servers_grouped['up_bytes'] / servers_grouped['down_bytes']
servers_grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()
servers_mean_ratios = servers_grouped.groupby('dst_ip')['ratio'].mean().reset_index()

In [75]:
servers_mean_ratios.sort_values(by="ratio",ascending=False)

,dst_ip,ratio
3,192.168.107.227,0.435827
4,192.168.107.234,0.434916
0,192.168.107.222,0.110127
1,192.168.107.223,0.109864
2,192.168.107.226,0.108967
5,192.168.107.238,0.108939


In [ ]:
servers_mean_ratios.to_csv("dst_ip_normal.csv")

In [76]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_internal_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET  and  ipaddress.IPv4Address(x['src_ip']) in NET  ,axis=1)]

private_dst_internal_normal.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [87]:
private_dst_internal_normal.groupby("src_ip")['dst_ip'].agg(list).reset_index().to_csv("normal_coms_internal.csv")

Bad pipe message: %s [b'0.9,image/avif,image/webp,*/*;q=0.8\r\nHost: localhost:37815\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64', b'rv:126.0) Gecko/20100101 Firefox/126.0\r\nAccept-Encoding: g']
Bad pipe message: %s [b'p, deflate, br, zstd\r\nAccept-Language: pt-PT,pt;q=0.8,en;q=0.5,en-US;q=0.3\r\nReferer: https://cuddly-fiesta-r6q57g7p4p7f5g', b'.github.dev/\r\nX-Request-ID: cfa53ff7b38952cee912947c2ef8194f\r\nX-Real-IP: 10.240.1.34\r\nX-Forwarded-Port', b'443\r\nX-Forwarded-Scheme: https\r\nX-Original-URI: /\r\nX-Sche']
Bad pipe message: %s [b'0.9,image/avif,image/webp,*/*;q=0.8\r\nHost: localhost:37815\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64', b'rv:126.0) Gecko/20100101 Firefox/126.0\r\nAccept-Encoding: g']
Bad pipe message: %s [b'p, deflate, br, zstd\r\nAccept-Language: pt-PT,pt;q=0.8,en;q=0.5,en-US;q=0.3\r\nReferer: https://cuddly-fiesta-r6q57g7p4p7f5g', b'.github.dev/\r\nX-Request-ID: d9721deff254e26c596ff82bd837475d\r\nX-Real-IP: 10.240.1.34\r\nX-For